In [5]:
import torch
from torch import nn as nn
import os

from torchvision import datasets, models, transforms
from torch import optim

In [6]:
ddir = '/content/drive/MyDrive/study/study by lecture/implementation/dataset/hym_data'

batch_size = 4
num_workers = 2

data_transformers = {
    'train' : transforms.Compose(
        [
          transforms.RandomResizedCrop(224),
          transforms.RandomHorizontalFlip(),
          transforms.ToTensor(),
          transforms.Normalize([0.490, 0.449, 0.411],[0.231, 0.221, 0.230])
        ]
    ),
    'val' : transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.490, 0.449, 0.411],[0.231, 0.221, 0.230])
        ]
    )    
}

image_data = {
    k : datasets.ImageFolder(os.path.join(ddir, k),data_transformers[k]) for k in ['train', 'val']
}

dloader = {
    k : torch.utils.data.DataLoader(image_data[k], batch_size = batch_size, shuffle = True, num_workers = num_workers) for k in ['train', 'val']
}

dset_size = {x : len(image_data[x]) for x in ['train', 'val']}
classes = image_data['train'].classes

dvc = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
class Bottleneck(nn.Module):
  def __init__(self, in_channels, growth_rate):
    super().__init()

    inner_channel = 4*growth_rate

    self.bottle_neck = nn.Sequential(
        nn.BatchNorm2d(in_channels),
        nn.ReLU(inplace = True),
        nn.Conv2d(in_channels, inner_channel, kernel_size = 1, bias = False),
        nn.BatchNorm2d(inner_channel),
        nn.ReLU(inplace = True),
        nn.Conv2d(inner_channel, growth_rate, kernel_size = 3, padding = 1, bias = False),
    )
  def forward(self, x):
    return torch.cat([x, self.bottle_neck(x)], 1)

    